## trie

In [ ]:
from typing import Dict
from queue import LifoQueue

In [ ]:
class TrieNode:

    def __init__(self, letter: str = "", parent: 'TrieNode' = None, depth: int = 0) -> None:
        super().__init__()
        self.letter: str = letter
        self.children: Dict[str, TrieNode] = dict()
        self.depth: int = depth
        self.parent: TrieNode = parent
        self.link: TrieNode = None

    def graft(self, text: str, sibling: 'TrieNode' = None) -> 'TrieNode':
        if len(text) == 0:
            return self
        if text[0] not in self.children:
            self.children[text[0]] = TrieNode(text[0], self, self.depth + 1)
        if sibling:
            sibling = sibling.children[text[0]]
            sibling.link = self.children[text[0]]
        return self.children[text[0]].graft(text[1:], sibling)

    def __contains__(self, item):
        if len(item) == 0:
            return True
        if not isinstance(item, str) or not item[0] in self.children.keys():
            return False
        return self.contains(item[1:])
        
    def __len__(self):
        length = 0
        for node in self.children.values():
            length += len(node)
        return length + 1


class Trie:

    def __init__(self, text: str) -> None:
        self.root: TrieNode = TrieNode()
        leaf = self.root.graft(text)
        self.root.children[text[0]].link = self.root
        for i in range(1, len(text)):
            head, sibling = self.up_link_down(leaf)
            if not head:
                sibling = self.root.children[text[i-1]]
                sibling.link = self.root
                head = self.root
            leaf = head.graft(text[i + head.depth:], sibling)

    def __contains__(self, item):
        return isinstance(item, str) and item in self.root
        
    def up_link_down(self, sibling: TrieNode) -> (TrieNode, TrieNode):
        letters = LifoQueue()
        while sibling and not sibling.link:
            letters.put(sibling.letter)
            sibling = sibling.parent
        if not sibling:
            return None, None
        node = sibling.link
        while not letters.empty():
            current_letter = letters.get()
            if current_letter in node.children.keys():
                node = node.children[current_letter]
                sibling = sibling.children[current_letter]
                sibling.link = node
            else:
                break
        return node, sibling

    def __len__(self):
        return len(self.root)

## suffix tree

In [ ]:

class SuffixTreeNode:

    def __init__(self, text: str, start: int = 0, end: int = 0, depth: int = 0, parent: 'SuffixTreeNode' = None) -> None:
        super().__init__()
        self.depth = depth
        self.start = start
        self.end = end
        self.full_text = text
        self.children: Dict[str, SuffixTreeNode] = dict()
        self.parent: SuffixTreeNode = parent
        self.link: SuffixTreeNode = None

    def graft(self, start):
        start = start + self.depth
        text = self.full_text[start:]
        child = SuffixTreeNode(self.full_text, start, len(self.full_text), self.depth + len(text), self)
        self.children[text[0]] = child

    def break_path(self, text: str):
        length = len(text)
        child = self.children[text[0]]
        new_node = SuffixTreeNode(self.full_text, child.start, child.start + length, self.depth + length, self)
        child.start = child.start + length
        child.parent = new_node
        new_node.children[child.label[0]] = child
        self.children[text[0]] = new_node
        return new_node

    def fast_find(self, text: str):
        child = self.children[text[0]]
        if len(child.label) < len(text):
            return child.fast_find(text[len(child.label):])
        elif len(child.label) == len(text):
            return child
        else:
            return self.break_path(text)

    def slow_find(self, text: str) -> 'SuffixTreeNode':
        if len(text) == 0 or text[0] not in self.children.keys():
            return self
        child = self.children[text[0]]
        for i in range(1, len(child.label)):
            if child.label[i] != text[i]:
                return self.break_path(text[:i])
        return child.slow_find(text[len(child.label):])

    @property
    def label(self):
        return self.full_text[self.start:self.end]

    def __contains__(self, item):
        if len(item) == 0:
            return True
        if not isinstance(item, str) or item[0] not in self.children:
            return False
        child = self.children[item[0]]
        for i in range(1, min(len(child.label), len(item))):
            if child.label[i] != item[i]:
                return False
        return len(item) < len(child.label) or item[len(child.label):] in child

    def __repr__(self) -> str:
        return f"[{self.start}:{self.end}] {self.full_text[self.start:self.end]}"


class SuffixTree:

    def __init__(self, text: str) -> None:
        self.root: SuffixTreeNode = SuffixTreeNode(text)
        self.root.graft(0)
        for i in range(1, len(text)):
            head= self.root.slow_find(text[i:])
            head.graft(i)

    def __contains__(self, item):
        return isinstance(item, str) and item in self.root